In [1]:
import numpy as np

import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()

# for the symbolic manipulation of jacobian
import sympy as sp
# from sympy import symbols
# from sympy import sin, cos, asin, acos, pi, atan2, sqrt
from sympy.utilities.lambdify import lambdify
# from sympy import Matrix

from scipy.optimize import minimize
from scipy.optimize import fsolve

import time

import odrive
import odrive.utils
import odrive.enums

## Connect to ODrive

In [ ]:
odrv0 = odrive.find_any()
if odrv0 is not None:
    print('Connected!')
    print('Odrive serial {}'.format(odrv0.serial_number))
    
    m0 = odrv0.axis0.motor.is_calibrated
    m1 = odrv0.axis1.motor.is_calibrated
    
    print('Motor 0 calibrated: {}'.format(m0))
    print('Motor 1 calibrated: {}'.format(m1))    
    
else:
    print('Not connected')
    

In [2]:
def connect_odrive(serial):
    odrv0 = odrive.find_any(serial_number = serial, timeout = 20)
    if odrv0 is not None:
        print('Connected!')
        print('Target serial {} \t Odrive serial {}'.format(serial, odrv0.serial_number))
        
        m0 = odrv0.axis0.motor.is_calibrated
        m1 = odrv0.axis1.motor.is_calibrated

        print('Motor 0 calibrated: {}'.format(m0))
        print('Motor 1 calibrated: {}'.format(m1))    
    else:
        print("ODrive ({}) not found".format(serial))
    return odrv0
    
odrv0 = connect_odrive('206A33A5304B')


Connected!
Target serial 206A33A5304B 	 Odrive serial 35640505086027
Motor 0 calibrated: True
Motor 1 calibrated: True


In [9]:
%qtconsole

## If not calibrated already perform calibration

In [3]:
odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE

time.sleep(15)

print('\t Motor 0 calibration result: {} \r\n'.format(odrv0.axis0.motor.is_calibrated), 
      '\t Motor 1 calibration result: {}'.format(odrv0.axis1.motor.is_calibrated))


	 Motor 0 calibration result: True 
 	 Motor 1 calibration result: True


In [4]:
%qtconsole

## Test position control mode

In [4]:
odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL

odrv0.axis0.controller.set_pos_setpoint(0,0,0)
odrv0.axis1.controller.set_pos_setpoint(0,0,0)

## Put back into idle mode

In [5]:
odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_IDLE
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_IDLE

## (can now put legs on if you want to)

## Define a get joint angles function 

In [46]:
motor_cpr = (odrv0.axis0.encoder.config.cpr, 
             odrv0.axis1.encoder.config.cpr)

def get_joints(odrv, cpr = motor_cpr, zero_pos = (0,0)):
    m0 = 2*np.pi*(odrv.axis0.encoder.pos_estimate - zero_pos[0])/motor_cpr[0]
    m1 = 2*np.pi*(odrv.axis1.encoder.pos_estimate - zero_pos[1])/motor_cpr[1]
    
    return (m0, m1)

print(get_joints(odrv0))

(0.006854736727298959, 0.0007194285499439664)


## Inspect joint angle measurement process

In [51]:
m0 = []
m1 = []

start = time.time()

t_curr = 0

while(t_curr < 20):
    t_curr = (time.time() - start);
    
    pos = get_joints(odrv0)
    m0.append(pos[0])
    m1.append(pos[1]) 
    
    plt.cla()
    plt.plot(np.array(m0)/np.pi)
    plt.plot(np.array(m1)/np.pi)
    plt.title('{0:.2f} seconds'.format(t_curr))
    plt.show()
    
    plt.pause(0.1)


## Configure output limits

In [53]:
curr_limit = 2 #A
odrv0.axis0.motor.config.current_lim = curr_limit
odrv0.axis1.motor.config.current_lim = curr_limit


## Def a function to send motor current commands 

In [54]:
def motor_get_current(odrv):
    return(odrv.axis0.motor.current_control.Iq_measured, 
           odrv.axis1.motor.current_control.Iq_measured)

def motor_set_current(odrv, current, current_limit = (curr_limit, curr_limit)):
    
    current_0 = min(np.abs(current[0]), current_limit[0]) * np.sign(current[0])
    current_1 = min(np.abs(current[1]), current_limit[1]) * np.sign(current[1])    
    
    odrv.axis0.controller.set_current_setpoint(current_0)
    odrv.axis1.controller.set_current_setpoint(current_1)
    
    return (current_0, current_1)
    
    

## Apply torques in one direction for a short time then reverse in opposite direction then kill motors

In [55]:
odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL

motor_set_current(odrv0, (.4, -.4))
time.sleep(0.35)
print(motor_get_current(odrv0))
motor_set_current(odrv0, (-.4, .4))
time.sleep(0.35)
print(motor_get_current(odrv0))
motor_set_current(odrv0, (0, 0))
time.sleep(0.25)
print(motor_get_current(odrv0))


odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_IDLE
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_IDLE

(0.31622374057769775, -0.169110506772995)
(-0.48017850518226624, 0.41434377431869507)
(-0.012473955750465393, 0.28185588121414185)


In [56]:
del odrv0

In [2]:
%qtconsole